##### Converting the predictions at day level.

In [0]:
def day_level_result(Asset_list,Raw_data_path,RN,NR,features_to_test,model_path,Anomaly_threshold):
  
  '''This function loads the predictions and then aggregate the trace level results into day level. Anomaly threshold column is created which tells the       percentage of traces marked as anomalous in a particular day. Finally, based on Anomaly_threshold any day is either marked as anomalous or             normal
  
  Input :: Pandas dataframe containing the predictions of random forest against all the traces either 0 or 1. Anomaly_threshold is also the input.
  Output :: Pandas dataframe containing the aggregated predictions of Random forest at day level and  prediction based on Anomaly threshold.
  '''
  
  model_result = predictions_at_trace_level(Asset_list,Raw_data_path,RN,NR,features_to_test,model_path)
  ### Calling the predictions_at_trace_level function from the previous file
  
  data_iso_forest_withCount =  model_result.groupby(['Name','DATE','forest_Predictions']).forest_Predictions.agg('count').to_frame('Anomaly_Count').reset_index()
  ## Grouping and aggregating the trace level result and creating anomaly_count column
  
  Result_df_noanomaly = data_iso_forest_withCount[data_iso_forest_withCount.forest_Predictions ==1]
  ## Selecting only the count of normal traces in a day
  
  noanomaly =Result_df_noanomaly[['Name','DATE','Anomaly_Count']]
  noanomaly.columns =['Name','DATE','NoAnomaly_Count']

  Result_df_anomaly = data_iso_forest_withCount[data_iso_forest_withCount.forest_Predictions==0]
  ## Selecting only the count of anomalous traces in a day
  anomaly =Result_df_anomaly[['Name','DATE','Anomaly_Count']]
  anomaly.columns =['Name','DATE','YesAnomaly_Count']

  Temp =data_iso_forest_withCount.merge(anomaly, on = ['Name','DATE'],how='left')
  Result_df_final =Temp.merge(noanomaly, on = ['Name','DATE'],how='left')
  ## Joining the above two dataframes to get anomalous trace count and normal trace count against each date
  
  Result_df_final.YesAnomaly_Count.fillna(0, inplace=True)
  Result_df_final.NoAnomaly_Count.fillna(0, inplace=True)
  ## Replacing the NA's with 0
  
  Result_df_final['Anomaly_percentage'] = (((Result_df_final['YesAnomaly_Count'])/(Result_df_final['YesAnomaly_Count']+Result_df_final['NoAnomaly_Count'])))*100
  ### Calculating the Percentage of anomalous traces for each day.

  Result_df_final =Result_df_final.drop_duplicates(['Name','DATE','YesAnomaly_Count','NoAnomaly_Count'])
  ### Dropping duplicates
  
  Result_df_final = Result_df_final.drop(columns = ['forest_Predictions','Anomaly_Count'])
  ### Dropping extra columns
  
  Result_df_final = Result_df_final.sort_values(['Name','DATE'])
  ## Sorting based on Asset name and Date
  
  
  ### Creating a new column y based on Anomaly_percentage. 1 means the day has been marked as anomalous and 0 means the day has been marked as normal
  if Anomaly_threshold == 0.0:

      # For 0 Anamoly Percentage

      Result_df_final.loc[Result_df_final['YesAnomaly_Count'] >= 1.0, 'y'] = 1

      Result_df_final['y'] = Result_df_final['y'].fillna(0)

      #For Anamoly Percentage

  else:

      Result_df_final.loc[Result_df_final['Anomaly_percentage'] >= Anomaly_threshold, 'y'] = 1

      Result_df_final['y'] = Result_df_final['y'].fillna(0)
  
  return Result_df_final
  